# Chapter 4 — AsyncIO Examples
This notebook collects runnable snippets from the chapter. Some cells (like `aiohttp`) may require extra packages or internet access.

## Hello AsyncIO

In [ ]:
import asyncio

async def main():
    print('Awaiting for ...')
    await asyncio.sleep(1)
    print('... AsyncIO!')

if __name__ == "__main__":
    asyncio.run(main())


## Coroutines and create_task

In [ ]:
import asyncio

async def other():
    print("I am a coroutine")

async def main():
    task = asyncio.create_task(other())
    print('Awaiting for ...')
    await asyncio.sleep(1)
    await task
    print('... AsyncIO!')

if __name__ == "__main__":
    asyncio.run(main())


## Concurrent vs sequential

In [ ]:
import asyncio, time

async def other(i, t):
    await asyncio.sleep(t)
    print(f"I am a coroutine {i}")

async def main():
    t1 = time.perf_counter()
    task1 = asyncio.create_task(other(1, 10))
    task2 = asyncio.create_task(other(2, 4))
    task3 = asyncio.create_task(other(3, 1))
    await task1
    await task2
    await task3
    t2 = time.perf_counter()
    print(f"Elapsed time {t2 - t1:.3f} s")

if __name__ == "__main__":
    asyncio.run(main())


## Sequential contrast

In [ ]:
import asyncio, time

async def other(i, t):
    await asyncio.sleep(t)
    print(f"I am a coroutine {i}")

async def main():
    t1 = time.perf_counter()
    await other(1, 10)
    await other(2, 4)
    await other(3, 3)
    t2 = time.perf_counter()
    print(f"Elapsed time {t2 - t1:.3f} s")

if __name__ == "__main__":
    asyncio.run(main())


## Task names & current_task()

In [ ]:
import asyncio

async def other(t):
    await asyncio.sleep(t)
    task = asyncio.current_task()
    print("I am a coroutine", task.get_name())

async def main():
    task1 = asyncio.create_task(other(10), name="1")
    task2 = asyncio.create_task(other(4), name="2")
    task3 = asyncio.create_task(other(3), name="3")
    await task1; await task2; await task3

if __name__ == "__main__":
    asyncio.run(main())


## gather returns

In [ ]:
import asyncio

async def coroutine(t, id):
    await asyncio.sleep(t)
    print(f"I am the coroutine {id}")
    return t + 2

async def main():
    results = await asyncio.gather(
        coroutine(10, "A"),
        coroutine(4, "B"),
        coroutine(2, "C"),
    )
    print("The results are:", results)

if __name__ == "__main__":
    asyncio.run(main())


## Future basics

In [ ]:
import asyncio

async def get_result(future: asyncio.Future):
    await asyncio.sleep(2)
    future.set_result('...a future result')

async def main():
    my_future = asyncio.Future()
    task = asyncio.create_task(get_result(my_future))
    await task
    print("I'm waiting for ...")
    print(await my_future)
    print('Before continuing with my execution')

if __name__ == "__main__":
    asyncio.run(main())


## Explicit event loop

In [ ]:
import asyncio

async def main():
    print('Starting...')
    await asyncio.sleep(3)
    print('...Ending')

if __name__ == "__main__":
    loop = asyncio.new_event_loop()
    asyncio.set_event_loop(loop)
    try:
        loop.run_until_complete(main())
    finally:
        loop.close()


## Async generator and async for

In [ ]:
import asyncio

async def gen(n):
    for i in range(n):
        await asyncio.sleep(0.2)
        yield i

async def main():
    async for i in gen(10):
        print(i)

if __name__ == "__main__":
    asyncio.run(main())


## as_completed

In [ ]:
import asyncio

async def f(i):
    print(f'start iteration step {i}')
    await asyncio.sleep(1)
    print(f'end iteration step {i}')
    return i

async def main():
    for coro in asyncio.as_completed([f(i) for i in range(10)]):
        result = await coro
        print('result received:', result)

if __name__ == "__main__":
    asyncio.run(main())


## Queue producer-consumer

In [ ]:
import asyncio
import random

async def producer(name, queue):
    n = random.randint(0, 5)
    await asyncio.sleep(n)
    await queue.put(n)
    print(f"Producer {name} adds {n} to the queue")

async def consumer(name, queue):
    while True:
        n = await queue.get()
        await asyncio.sleep(n)
        print(f"Consumer {name} receives {n} from the queue")
        queue.task_done()

async def main(nproducers=4, nconsumers=2):
    q = asyncio.Queue()
    producers = [asyncio.create_task(producer(n, q)) for n in range(nproducers)]
    consumers = [asyncio.create_task(consumer(n, q)) for n in range(nconsumers)]
    await asyncio.gather(*producers)
    await q.join()
    for c in consumers:
        c.cancel()

if __name__ == "__main__":
    asyncio.run(main())


## run_in_executor (heavy)

In [ ]:
import asyncio
import time

def heavy_computation(n):
    start = time.time()
    s = 0
    for i in range(n):
        s += i * i
    end = time.time()
    print(f"Computation took {end - start:.2f} seconds")
    return s

async def main():
    loop = asyncio.get_running_loop()
    result = await loop.run_in_executor(None, heavy_computation, 10**7)
    print("Result:", result)

if __name__ == "__main__":
    asyncio.run(main())


## Error handling & gather(return_exceptions)

In [ ]:
import asyncio

async def fetch_data(n):
    if n == 2:
        raise ValueError("Error in task 2")
    await asyncio.sleep(1)
    return f"Task {n} completed"

async def main():
    tasks = [fetch_data(i) for i in range(5)]
    results = await asyncio.gather(*tasks, return_exceptions=True)
    for i, result in enumerate(results):
        if isinstance(result, Exception):
            print(f"Task {i} generated an error: {result}")
        else:
            print(f"Task result {i}: {result}")

if __name__ == "__main__":
    asyncio.run(main())


## Retry, timeout, logging

In [ ]:
import asyncio, random, logging

logging.basicConfig(level=logging.INFO, format='%(asctime)s - %(levelname)s - %(message)s')

async def fetch_data_with_retry(n, retries=3, delay=0.5):
    for attempt in range(retries):
        try:
            if random.random() < 0.5:
                raise ValueError(f"Temporary error in task {n}")
            await asyncio.sleep(0.3)
            logging.info(f"Task {n} successfully completed on attempt {attempt + 1}")
            return f"Result of task {n}"
        except ValueError as e:
            logging.warning(f"Attempt {attempt + 1} for task {n} failed: {e}")
            await asyncio.sleep(delay)
    raise RuntimeError(f"Task {n} failed after {retries} attempts")

async def fetch_data(n):
    await asyncio.sleep(n * 0.2)
    return f"Task {n} completed"

async def main():
    # Retry demo
    tasks = [fetch_data_with_retry(i) for i in range(5)]
    results = await asyncio.gather(*tasks, return_exceptions=True)
    logging.info(f"Results: {results}")
    # Timeout demo
    timed = [asyncio.wait_for(fetch_data(i), timeout=0.5) for i in range(5)]
    results2 = await asyncio.gather(*timed, return_exceptions=True)
    for i, r in enumerate(results2):
        if isinstance(r, asyncio.TimeoutError):
            logging.error(f"Task {i} expired")
        else:
            logging.info(f"Task result {i}: {r}")

if __name__ == "__main__":
    asyncio.run(main())


## Patterns: rate limiter

In [ ]:
import asyncio, random

async def fetch_data(n, limiter):
    async with limiter:
        await asyncio.sleep(random.uniform(0.1, 0.4))
        print(f"Task {n} completed")
        return n

async def main():
    rate_limiter = asyncio.Semaphore(3)
    tasks = [fetch_data(i, rate_limiter) for i in range(10)]
    results = await asyncio.gather(*tasks)
    print("Results with rate limiting:", results)

if __name__ == "__main__":
    asyncio.run(main())


## Patterns: circuit breaker

In [ ]:
import asyncio

class CircuitBreaker:
    def __init__(self, max_failures, reset_timeout):
        self.max_failures = max_failures
        self.reset_timeout = reset_timeout
        self.failures = 0
        self.open = False

    async def call(self, func, *args):
        if self.open:
            print("Open circuit, attempt refused.")
            return
        try:
            result = await func(*args)
            self.failures = 0
            return result
        except Exception as e:
            self.failures += 1
            print(f"Error: {e}, failed attempts: {self.failures}")
            if self.failures >= self.max_failures:
                self.open = True
                print("Open circuit, waiting for reset.")
                await asyncio.sleep(self.reset_timeout)
                self.failures = 0
                self.open = False

async def unstable_task(n):
    if n % 2 == 0:
        raise ValueError("Simulated Error")
    await asyncio.sleep(0.1)
    return f"Task {n} completed"

async def main():
    cb = CircuitBreaker(max_failures=2, reset_timeout=1)
    tasks = [cb.call(unstable_task, i) for i in range(6)]
    await asyncio.gather(*tasks)

if __name__ == "__main__":
    asyncio.run(main())


## Patterns: bulkhead

In [ ]:
import asyncio

async def limited_task(n, limiter, section_name):
    async with limiter:
        await asyncio.sleep(0.2)
        print(f"Task {n} completed in section {section_name}")

async def main():
    limiter_a = asyncio.Semaphore(2)
    limiter_b = asyncio.Semaphore(3)
    tasks = [limited_task(i, limiter_a, 'A') for i in range(5)] + \            [limited_task(i, limiter_b, 'B') for i in range(5, 10)]
    await asyncio.gather(*tasks)

if __name__ == "__main__":
    asyncio.run(main())
